In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/07 21:05:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Question 1:

In [3]:
spark.version

'3.5.5'

In [ ]:
# Question 2:

In [ ]:
# !mkdir -p data/raw/yellow/2024/10

In [7]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet -O "data/raw/yellow/2024/10/yellow_tripdata_2024-10.parquet"

--2025-03-07 21:08:14--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.6, 18.155.128.222, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘data/raw/yellow/2024/10/yellow_tripdata_2024-10.parquet’

data/raw/yellow/202 100%[===================>]  61.36M   177MB/s    in 0.3s    

2025-03-07 21:08:15 (177 MB/s) - ‘data/raw/yellow/2024/10/yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [17]:
df_yellow = spark.read \
    .option("header", "true") \
    .parquet('data/raw/yellow/2024/10/*')

In [19]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [22]:
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         162|         246|           1|       18.4|  1.0|    0.5|       1.

In [12]:
df_yellow \
    .repartition(4) \
    .write.parquet('data/pq/yellow/2024/10/')

In [13]:
# Question 2:
# ~/notebooks/data/pq/yellow/2024/10$ ls -lh
# 23MB
Answer : 25MB

In [20]:
# Question 3:

In [24]:
df_yellow.createOrReplaceTempView("yellow_trips_data")

In [25]:
spark.sql("""
SELECT
    count(1)
FROM 
    yellow_trips_data
WHERE 
    DATE(tpep_pickup_datetime) = DATE('2024-10-15')
""").show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



In [ ]:
# Answer : 125,567

In [ ]:
# Question 4:

In [27]:
from pyspark.sql.functions import col, unix_timestamp

In [28]:
# Calculate trip duration in hours
df_yellow_with_duration = df_yellow.withColumn(
    "trip_duration_hours",
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600
)

In [29]:
df_yellow_with_duration.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee| trip_duration_hours|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+
|       2| 2024-10-01 00:30:44|  2024-10-01 00:48:26|              1|          3.0|         1|                 N|         16

In [31]:
df_yellow_with_duration.createOrReplaceTempView("yellow_duration_trips_data")

In [34]:
spark.sql("""
SELECT
    trip_duration_hours
FROM 
    yellow_duration_trips_data
ORDER BY 
    trip_duration_hours DESC
LIMIT 10
""").show()

+-------------------+
|trip_duration_hours|
+-------------------+
| 162.61777777777777|
|            143.325|
| 137.76055555555556|
| 114.83472222222223|
|  89.89833333333333|
|  89.44611111111111|
|  70.29916666666666|
|  67.57333333333334|
|  66.06666666666666|
|            46.4225|
+-------------------+



In [ ]:
# Answer: 162

In [ ]:
# Question 6:

In [36]:
!mkdir -p data/raw/taxi_zone_lookup

In [37]:
!wget wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv -O "data/raw/taxi_zone_lookup/taxi_zone_lookup.csv"

--2025-03-07 21:33:58--  http://wget/
Resolving wget (wget)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘wget’
--2025-03-07 21:33:58--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.155.128.222, 18.155.128.187, 18.155.128.46, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.155.128.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘data/raw/taxi_zone_lookup/taxi_zone_lookup.csv’

data/raw/taxi_zone_ 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-07 21:33:58 (34.6 MB/s) - ‘data/raw/taxi_zone_lookup/taxi_zone_lookup.csv’ saved [12331/12331]

FINISHED --2025-03-07 21:33:58--
Total wall clock time: 0.07s
Downloaded: 1 files, 12K in 0s (34.6 MB/s)


In [38]:
df_taxi_zone = spark.read \
        .option("header", "true") \
        .csv('data/raw/taxi_zone_lookup/*')

In [39]:
df_taxi_zone.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [49]:
df_taxi_zone.createOrReplaceTempView("taxi_zone")

In [52]:
spark.sql("""
SELECT DISTINCT
    Zone
FROM 
    taxi_zone
""").show()

+--------------------+
|                Zone|
+--------------------+
|Governor's Island...|
|           Homecrest|
|              Corona|
|    Bensonhurst West|
|         Westerleigh|
|      Newark Airport|
|Charleston/Totten...|
|          Douglaston|
|East Concourse/Co...|
|          Mount Hope|
|      Pelham Parkway|
|         Marble Hill|
|           Rego Park|
|       Dyker Heights|
|Heartland Village...|
|Upper East Side S...|
|   Kew Gardens Hills|
|       Rikers Island|
|             Bayside|
|     Jackson Heights|
+--------------------+
only showing top 20 rows



In [ ]:
spark.sql("""
SELECT 
    Zone
FROM 
    taxi_zone
""").show()

In [40]:
df_joined = df_taxi_zone.join(df_yellow, df_taxi_zone.LocationID == df_yellow.PULocationID, "inner")

In [41]:
df_joined.show()

+----------+---------+--------------------+------------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|LocationID|  Borough|                Zone|service_zone|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+----------+---------+--------------------+------------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       162|Manhattan|

In [42]:
df_joined.createOrReplaceTempView("yellow_pickup_zone_data")

In [54]:
spark.sql("""
SELECT
    Zone,
    count(1) as trip_count
FROM 
    yellow_pickup_zone_data
WHERE 
    service_zone = 'Yellow Zone'
GROUP BY
    Zone
ORDER BY 
    trip_count
""").show()

+--------------------+----------+
|                Zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
|     Randalls Island|       180|
|        Battery Park|      1452|
|Stuy Town/Peter C...|      5701|
|Two Bridges/Sewar...|      5847|
|           Chinatown|      5878|
|       Alphabet City|      6781|
|             Seaport|      8631|
|        Bloomingdale|      9655|
|Financial Distric...|     11851|
|           Hudson Sq|     19815|
|  World Trade Center|     20498|
|Financial Distric...|     23044|
|        Clinton West|     23714|
|   Battery Park City|     25091|
|    Manhattan Valley|     29491|
|                SoHo|     31967|
| Little Italy/NoLiTa|     38512|
|            Kips Bay|     39058|
|     Lower East Side|     39294|
+--------------------+----------+
only showing top 20 rows



In [ ]:
# Answer: Governor's Island/Ellis Island/Liberty Island